In [1]:
import torch
import clip
from PIL import Image
import numpy as np
import glob
import math
from scipy import spatial
import os
from pathlib import Path
import sys
import time
from tqdm import tqdm

/home/toonies/miniconda3/envs/aic/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
ROOT = Path(os.getcwd()).resolve()

if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

WORK_DIR = ROOT.parents[0]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, preprocess = clip.load("ViT-B/32", device=device)

print("Device :", device)
print("Current Working Directory:", WORK_DIR)

Device : cuda
Current Working Directory: /home/toonies/Learn/Text_Video_Retrieval


In [6]:
des_path =  f"{WORK_DIR}/data/dicts/npy_clip"
paths = f"{WORK_DIR}/data/images"


for keyframe in tqdm(os.listdir(paths)):
  path_keyframe = os.path.join(paths,keyframe)
  video_paths = sorted(glob.glob(f"{path_keyframe}/*/"))
  video_paths = ['/'.join(i.split('/')[:-1]) for i in video_paths]

  start_time = time.time()
  for vd_path in video_paths:

    re_feats = []
    keyframe_paths = glob.glob(f'{vd_path}/*.jpg')
    keyframe_paths = sorted(keyframe_paths, key=lambda x : x.split('/')[-1].replace('.jpg',''))

    for keyframe_path in keyframe_paths:
      image = preprocess(Image.open(keyframe_path)).unsqueeze(0).to(device)

      with torch.no_grad():
          image_feats = model.encode_image(image)

      image_feats /= image_feats.norm(dim=-1, keepdim=True)
      image_feats = image_feats.detach().cpu().numpy().astype(np.float16).flatten() 

      re_feats.append(image_feats)

    name_npy = vd_path.split('/')[-1]
    outfile = f'{des_path}/{name_npy}.npy'
    np.save(outfile, re_feats)

100%|██████████| 2/2 [04:42<00:00, 141.02s/it]
